In [139]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KDTree

rng = np.random.RandomState(0)

In [145]:
df = pd.read_csv('Laptop-Preise.csv', sep=';', decimal=',')
df = df[df.extern_Schnittstellen != 2300] # Ausreißer löschen
df = df.reset_index(drop=True) # Reset Index
df.drop(['Betriebssystem_OHNE', 'Betriebssystem_Mac', 'Marke_Dell'], axis=1) # Mac und Marke_Apple sind identische Merkmale
df_noPrice = df.drop('Preis', axis=1)
error_mse = pd.DataFrame(columns=['chance', 'LM', 'KNN_1', 'KNN_3', 'KNN_5'])
error_stdErr = pd.DataFrame(columns=['chance', 'LM', 'KNN_1', 'KNN_3', 'KNN_5'])

In [141]:
def del_ran(df, chance):
    rand_array = np.random.rand(df.shape[0])
    delete_entries = rand_array < chance
    keep_entries = rand_array >= chance
    return {'keep' : df.iloc[keep_entries], 'delete' : df.iloc[delete_entries]}

In [158]:
temp = del_ran(df_noPrice, 0.01)
# temp['keep']
# temp['delete']

,Kerne,Takt_Prozessor,Arbeitsspeicher,Bild_Display,Speicher_Grafik,extern_Schnittstellen,HDD,SSD,Akku_Kapazitaet,PPI_Display,...,Marke_Acer,Marke_Apple,Marke_Dell,Marke_Lenovo,Betriebssystem_Windows,Betriebssystem_Mac,Betriebssystem_DOS,Betriebssystem_LINUX,Betriebssystem_OHNE,Mikrofon_Sound_vorhanden
79,4,1400,8,13.3,0,3,0,128,58.2,226,...,0,1,0,0,0,1,0,0,0,1
152,4,1600,8,13.3,0,6,0,512,50.0,165,...,0,0,0,0,1,0,0,0,0,1
452,2,2300,8,15.6,0,5,0,256,30.0,141,...,0,0,0,1,1,0,0,0,0,0
645,4,1600,8,14.0,0,6,0,256,45.0,157,...,0,0,0,1,1,0,0,0,0,1
730,2,2300,4,17.3,0,6,0,256,41.0,106,...,0,0,0,0,0,0,1,0,0,0
750,6,2600,32,15.4,4096,5,0,512,83.6,220,...,0,1,0,0,0,1,0,0,0,1
924,6,2600,32,15.4,4096,5,0,512,83.6,220,...,0,1,0,0,0,1,0,0,0,1
1018,6,2600,16,15.6,6144,6,1000,256,51.0,141,...,0,0,1,0,1,0,0,0,0,0


In [196]:
k = temp['keep']
d = temp['delete']
t = d.iloc[0].values.reshape(1, -1)

print(len(t))
print(k.shape)

tree = KDTree(temp['keep'], leaf_size=5)              
dist, ind = tree.query(t, k=3)

print

# for t in d.iterrows():
#     dist, ind = tree.query(t, k=3)
#     print(ind, dist)


1
(1030, 32)


In [54]:
tree = KDTree(df, leaf_size=5)              
dist, ind = tree.query(df[:1], k=5) # Bei den Queries wird der Index 1 als Startindex verwendet? --> Nein
print(ind)  # indices of 3 closest neighbors (Indizes bei 0 beginnend)
print(dist)

[[   0  281  227  115 1028]]
[[  0.          61.05644929 100.01499888 100.98955392 101.30429408]]


In [52]:
df[:1]

,Preis,Kerne,Takt_Prozessor,Arbeitsspeicher,Bild_Display,Speicher_Grafik,extern_Schnittstellen,HDD,SSD,Akku_Kapazitaet,...,Marke_Acer,Marke_Apple,Marke_Dell,Marke_Lenovo,Betriebssystem_Windows,Betriebssystem_Mac,Betriebssystem_DOS,Betriebssystem_LINUX,Betriebssystem_OHNE,Mikrofon_Sound_vorhanden
0,299.0,2,2600,8,15.6,0,6,0,256,41.0,...,0,0,0,0,0,0,0,0,1,1


In [55]:
df.iloc[ind[0]]


,Preis,Kerne,Takt_Prozessor,Arbeitsspeicher,Bild_Display,Speicher_Grafik,extern_Schnittstellen,HDD,SSD,Akku_Kapazitaet,...,Marke_Acer,Marke_Apple,Marke_Dell,Marke_Lenovo,Betriebssystem_Windows,Betriebssystem_Mac,Betriebssystem_DOS,Betriebssystem_LINUX,Betriebssystem_OHNE,Mikrofon_Sound_vorhanden
0,299.0,2,2600,8,15.6,0,6,0,256,41.0,...,0,0,0,0,0,0,0,0,1,1
282,349.0,2,2600,8,17.3,0,6,0,256,41.0,...,0,0,0,0,0,0,0,0,1,1
228,399.0,2,2600,8,15.6,0,6,0,256,41.0,...,0,0,0,0,1,0,0,0,0,0
116,399.0,2,2600,8,17.3,0,6,0,256,41.0,...,0,0,0,0,0,0,0,0,1,1
1029,399.0,2,2600,8,14.0,0,6,0,256,41.0,...,0,0,0,0,0,0,1,0,0,0
